In [4]:
! pip install tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=26431f9854bb86ddd90dd2210b8da65375906ef1636bdbefb0004a79aa30abe6
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: numpy
    Found

In [5]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from official.nlp import optimization

import matplotlib.pyplot as plt

import shutil

In [6]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')


84125825/84125825 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [12]:
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
train_dir = os.path.join(dataset_dir, 'train')
print(train_dir)

./aclImdb/train


In [13]:
remove_dir = os.path.join(train_dir, 'unsup')
#shutil.rmtree(remove_dir)
print(remove_dir)

./aclImdb/train/unsup


In [21]:
! ls

aclImdb  aclImdb_v1_extracted  aclImdb_v1.tar.gz  sample_data


In [22]:
AUTOTUNE = tf.data.AUTOTUNE

batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 0 files belonging to 1 classes.
Using 0 files for training.


ValueError: No text files found in directory aclImdb/train. Allowed format: .txt

In [23]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [24]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [25]:
text_test = ['i love that movie!!']
text_preprocessed = bert_preprocess_model(text_test)

print(list(text_preprocessed.keys()))

['input_type_ids', 'input_mask', 'input_word_ids']


In [26]:
bert_results = bert_model(text_preprocessed)

In [27]:
print(bert_results)

{'pooled_output': <tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 7.76454866e-01,  9.91110444e-01, -2.59568840e-01,
         3.30063760e-01,  8.09618354e-01,  9.20033216e-01,
         9.93348360e-01, -8.98253262e-01, -2.35161647e-01,
        -9.99105096e-01,  8.97238627e-02, -9.60274637e-01,
        -9.70800340e-01, -9.97449934e-01,  9.78558883e-02,
         5.09063900e-01,  1.20897539e-01,  3.95408541e-01,
        -1.99715361e-01, -7.24602044e-02,  1.61638781e-01,
         1.13170229e-01, -6.12717450e-01,  7.99563468e-01,
         9.98396516e-01,  9.60603237e-01, -4.88662153e-01,
        -4.47731912e-01,  2.83215046e-01,  3.77330743e-02,
         3.36269945e-01, -3.33704144e-01,  3.70335907e-01,
        -2.30938479e-01,  2.99133658e-01, -9.29268748e-02,
        -5.19890785e-01, -5.06099999e-01, -3.14370692e-01,
        -4.84367698e-01, -1.48667440e-01, -3.01012248e-01,
         9.33154449e-02, -7.14992523e-01, -1.08132429e-01,
        -6.57907486e-01, -9.93202031e-01,  5.58

In [28]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')

  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')

  outputs = encoder(encoder_inputs)

  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.2)(net)
  net = tf.keras.layers.Dense(1, name='classifier')(net)

  return tf.keras.Model(text_input, net)


In [30]:
classifier_model = build_classifier_model()
raw_result = classifier_model(tf.constant(text_test))

print(tf.sigmoid(raw_result))

ValueError: Exception encountered when calling layer 'preprocessing' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'preprocessing' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=False, name=text>
  • training=None

In [ ]:
classifier_model.summary()

In [ ]:
tf.keras.utils.plot_model(classifier_model)

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
epochs = 5

steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')


In [ ]:
classifier_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
history = classifier_model.fit(x=train_ds, validation_data=val_ds, epochs=epochs)

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

In [ ]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


In [ ]:
examples = [
    'i love that movie!!',
    'thas movie was okish',
    'i realy think that movie was bad',
    'i loved but was awful',
    'ok tier',
    'was awful but i loved',
    'i loved but i was too afraid'
]

results = tf.sigmoid(classifier_model(tf.constant(examples)))

In [ ]:
print_my_examples(examples, results)